In [3]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=2)
mp_drawing = mp.solutions.drawing_utils

# Load your earring image with transparency (RGBA)
earring_image = cv2.imread('Earring1.png', cv2.IMREAD_UNCHANGED)
jewelry_image = cv2.imread('jewellery.png',cv2.IMREAD_UNCHANGED)
# Function to overlay image
def overlay_image(background, overlay, pos, earing_size):
    x, y = pos
    y = y+ 120
    # x = x-10 
    overlay = cv2.resize(overlay, earing_size)

    # Ensure overlay image is in the correct format
    if overlay.shape[2] == 4:  # Check if overlay has an alpha channel
        overlay_image = overlay[..., :3]
        mask = overlay[..., 3:]
    else:
        overlay_image = overlay
        mask = np.ones(overlay_image.shape[:2], dtype=np.uint8) * 255  # Create a mask of all 255 (opaque)

    h, w, _ = overlay_image.shape
    roi = background[y:y+h, x:x+w]

    # Use alpha blending
    try:
        mask_inv = cv2.bitwise_not(mask)
        roi_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
        roi_fg = cv2.bitwise_and(overlay_image, overlay_image, mask=mask)

        background[y:y+h, x:x+w] = cv2.add(roi_bg, roi_fg)
    except:
        pass

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the image and get the face landmarks
    result = face_mesh.process(rgb_frame)
    
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            h, w, _ = frame.shape
            
            # Get landmark positions
            left_eye_outer = face_landmarks.landmark[33]
            right_eye_outer = face_landmarks.landmark[263]
            nose_tip = face_landmarks.landmark[1]
            
            # Convert normalized positions to pixel values
            left_eye_pos = (int(left_eye_outer.x * w), int(left_eye_outer.y * h))
            right_eye_pos = (int(right_eye_outer.x * w), int(right_eye_outer.y * h))
            nose_tip_pos = (int(nose_tip.x * w), int(nose_tip.y * h))

            # Calculate distance between eyes to determine the size of the earring
            half_face_distance = dist(left_eye_pos,nose_tip_pos)*0.5
            earing_size = (int(half_face_distance), int(half_face_distance))
            
            eye_distance = dist(left_eye_pos,right_eye_pos)*1.5
            overlay_size = (int(eye_distance), int(eye_distance))
            
            # Overlay the jewelry image
            overlay_image(frame, jewelry_image, (int(left_eye_outer.x * w)-20, int(left_eye_outer.y * h -(left_eye_outer.z+1)*30)), overlay_size)
            
            # Overlay the earring image
            overlay_image(frame, earring_image, (int(right_eye_outer.x * w)+10, int(right_eye_outer.y * h)-75), earing_size)
            overlay_image(frame, cv2.flip(earring_image,1), (int(left_eye_outer.x * w)-25, int(left_eye_outer.y * h)-75), earing_size)

            # Optionally draw the face mesh
            # mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)
    
    # Display the frame
    cv2.imshow('earring Filter', cv2.flip(frame,1))
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\dipak\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [2]:
import cv2
import mediapipe as mp
import numpy as np


In [1]:
def dist(a,b):
    dis = ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5
    return dis

In [ ]:
earring_image.shape

(307, 280, 4)